# Model recovery attack in split learning with multiple data owners

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
def make_dataset(X, Y, f):
    x = tf.data.Dataset.from_tensor_slices(X)
    y = tf.data.Dataset.from_tensor_slices(Y)
    x = x.map(f)
    xy = tf.data.Dataset.zip((x, y))
    xy = xy.shuffle(10000)
    return xy

df = pd.read_excel('../datasets/credit-card.xls', header=1, index_col=0).sample(frac=1)
min_values = df.drop(columns=["default payment next month"]).describe().transpose()['min'].to_numpy()
max_values = df.drop(columns=["default payment next month"]).describe().transpose()['max'].to_numpy()
x = df.drop(columns=["default payment next month"]).to_numpy()
x = (x-min_values)/(max_values-min_values)
y = df["default payment next month"].to_numpy().reshape((len(x), 1)).astype("float32")
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
attack_ds = make_dataset(x_train, y_train, lambda t: t)
train_ds = make_dataset(x_test, y_test, lambda t: t)

train_size = len(x_test)

2022-02-08 11:57:39.130163: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_SYSTEM_DRIVER_MISMATCH: system has unsupported display driver / cuda driver combination
2022-02-08 11:57:39.130222: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: cgpa4
2022-02-08 11:57:39.130236: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: cgpa4
2022-02-08 11:57:39.130384: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.47.3
2022-02-08 11:57:39.130418: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 495.46.0
2022-02-08 11:57:39.130428: E tensorflow/stream_executor/cuda/cuda_diagnostics.cc:313] kernel version 495.46.0 does not match DSO version 510.47.3 -- cannot find working devices in this configuration
2022-02-08 11:57:39.130987: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binar

In [3]:
num_classes = 2

def make_f(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.BatchNormalization()(xin)
    x = tf.keras.layers.Dense(32, activation="relu")(x)
    x = tf.keras.layers.Dense(64, activation="relu")(x)
    output = tf.keras.layers.Dense(128, activation="relu")(x)
    return tf.keras.Model(xin, output)

def make_g(input_shape, class_num):
    xin = tf.keras.layers.Input(input_shape)
    x = tf.keras.layers.Dense(256, activation="relu")(xin)
    # x = tf.keras.layers.Dropout(0.5)(x)
    output = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    return tf.keras.Model(xin, output)

input_shape = train_ds.element_spec[0].shape
f = make_f(input_shape)
intermediate_shape = f.layers[-1].output_shape[1:]
g = make_g(intermediate_shape, num_classes)

In [4]:
batch_size = 32
epoches = 5
# note that iterations is the number of batches we iterate
iterations = epoches * train_size // batch_size
learning_rate = 0.001

In [5]:
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
train_batches = train_ds.batch(batch_size=batch_size, drop_remainder=True).repeat(-1).take(iterations)
train_ref = []
z_ref = []

log = []
iter_count = 0
log_frequency = 50

for (x_batch, y_batch) in train_batches:
    
    with tf.GradientTape(persistent=True) as tape:
        z = f(x_batch, training=True)
        y_pred = g(z, training = True)
        if num_classes == 2:
            loss = tf.keras.losses.binary_crossentropy(y_true=y_batch, y_pred=y_pred)
            acc = tf.keras.metrics.binary_accuracy(y_batch, y_pred)
        else:
            loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y_batch, y_pred=y_pred)
            acc = tf.metrics.sparse_categorical_accuracy(y_batch, y_pred)
    var = f.trainable_variables + g.trainable_variables
    grad = tape.gradient(loss, var)
    optimizer.apply_gradients(zip(grad, var))

    iter_loss = sum(loss) / len(loss)
    iter_acc = sum(acc) / len(acc)
    log.append([iter_loss, iter_acc])
    iter_count += 1

    train_ref.append(x_batch)
    z_ref.append(z)

    if (iter_count - 1) % log_frequency == 0:
        print("Iteration %04d: Training loss: %0.4f training accuracy: %0.4f" % (iter_count, iter_loss, iter_acc))

Iteration 0001: Training loss: 0.6844 training accuracy: 0.5625
Iteration 0051: Training loss: 0.6012 training accuracy: 0.7188
Iteration 0101: Training loss: 0.5379 training accuracy: 0.7812
Iteration 0151: Training loss: 0.4945 training accuracy: 0.7500
Iteration 0201: Training loss: 0.5604 training accuracy: 0.7812
Iteration 0251: Training loss: 0.4603 training accuracy: 0.7812
Iteration 0301: Training loss: 0.5659 training accuracy: 0.6562
Iteration 0351: Training loss: 0.2693 training accuracy: 0.9062
Iteration 0401: Training loss: 0.4532 training accuracy: 0.7812
Iteration 0451: Training loss: 0.4237 training accuracy: 0.8125
Iteration 0501: Training loss: 0.5915 training accuracy: 0.7188
Iteration 0551: Training loss: 0.4750 training accuracy: 0.7812
Iteration 0601: Training loss: 0.3806 training accuracy: 0.8750
Iteration 0651: Training loss: 0.2940 training accuracy: 0.8438
Iteration 0701: Training loss: 0.4602 training accuracy: 0.8125
Iteration 0751: Training loss: 0.5520 tr

In [6]:
def make_generator(input_shape):
    xin = tf.keras.layers.Input(input_shape)
    act = "relu"
    x = tf.keras.layers.Dense(512, activation=act)(xin)
    x = tf.keras.layers.Dense(256, activation=act)(x)
    x = tf.keras.layers.Dense(128, activation=act)(x)
    x = tf.keras.layers.Dense(64, activation=act)(x)
    x = tf.keras.layers.Dense(32, activation=act)(x)
    x = tf.keras.layers.Dense(23, activation="sigmoid")(x)
    return tf.keras.Model(xin, x)

In [8]:
f_temp = tf.keras.models.clone_model(f)
f_temp.set_weights(f.get_weights())

generator = make_generator((256,))

x_opt = tf.keras.optimizers.Adam(learning_rate=0.001)
# f_opt = tf.keras.optimizers.Adam(learning_rate=0.0001)

# inference_batches = attack_ds.batch(batch_size=32, drop_remainder=True).repeat(-1).take(attack_iterations)

attack_iter_count = 0

log = []

# for (x_batch, y_batch) in inference_batches:
for i in range(100):

    f_opt = tf.keras.optimizers.Adam(learning_rate=0.0001)

    z = z_ref[iterations - i - 1]
    x = train_ref[iterations - i - 1]

    # x_temp = np.zeros_like(x.numpy())
    # x_temp.fill(0.5)
    # x_temp = tf.Variable(x_temp)

    for _ in range(100):

        for _ in range(50):
            with tf.GradientTape() as tape:
                # x_temp = generator(z, training=True)
                x_temp = generator(tf.concat([z, tf.constant(np.random.rand(*(z.numpy().shape)).astype("float32"))],1), training=True)
                loss_x = tf.keras.losses.MeanSquaredError()(f_temp(x_temp, training=False), z)
            vars = generator.trainable_variables
            grad = tape.gradient(loss_x, vars)
            x_opt.apply_gradients(zip(grad, vars))
            # loss = lambda: tf.keras.losses.MeanSquaredError()(f_temp(x_temp, training=False), z)
            # x_opt.minimize(loss, var_list=[x_temp])

        for _ in range(1):
            with tf.GradientTape() as tape:
                loss_f = tf.keras.losses.MeanSquaredError()(f_temp(x_temp, training=True), z)
            vars = f_temp.trainable_variables
            grad = tape.gradient(loss_f, vars)
            f_opt.apply_gradients(zip(grad, vars))

    attack_mse = tf.losses.MeanSquaredError()(x_temp, x)
    rg_uniform = tf.losses.MeanSquaredError()(x, np.random.rand(*(x.numpy().shape)))
    rg_normal = tf.losses.MeanSquaredError()(x, np.random.normal(0.5, 0.25, size=(x.numpy().shape)))
    log.append([rg_uniform, rg_normal, attack_mse])
    if (attack_iter_count % 10 == 0):
        print("Iteration %04d: RG: %0.4f reconstruction validation: %0.4f" % (attack_iter_count, rg_uniform, attack_mse))
    attack_iter_count += 1

Iteration 0000: RG: 0.2355 reconstruction validation: 0.0081


In [152]:
x_temp[0].numpy()

array([0.52637647, 0.90229411, 0.35989075, 0.60646825, 0.36117107,
       0.63459949, 0.4951778 , 0.52023932, 0.59983958, 0.37528527,
       0.46935235, 0.37547181, 0.80484384, 0.34316436, 0.44915073,
       0.45964282, 0.62598943, 0.43523487, 0.31866709, 0.43191344,
       0.36823661, 0.32591818, 0.27166362])

In [153]:
x[0].numpy()

array([0.25252525, 1.        , 0.16666667, 0.66666667, 0.15517241,
       0.2       , 0.2       , 0.2       , 0.2       , 0.2       ,
       0.2       , 0.3553174 , 0.27729314, 0.20441946, 0.36058407,
       0.25631306, 0.37157862, 0.00914428, 0.00473739, 0.00759676,
       0.00844928, 0.01228521, 0.00945777])